In [1]:
import json
import os
import openai

with open('keys.json', "r") as f:
    keys = json.load(f)
print(keys.keys())

openai.api_key = keys['secret']

with open("counterfact.json", "r") as f:
    counterfact = json.load(f)
if os.path.exists("related.json"):
    with open("related.json", "r") as f:
        related = json.load(f)
else:
    related = []
print(len(counterfact))
print(len(related))
totalTokens = 0

def costEstimate(tokens):
    return (tokens/1000) * 0.002


dict_keys(['secret'])
21919
21919


In [2]:
def genMsg(idx):
    msg = counterfact[idx]['requested_rewrite']['prompt']
    msg = msg.replace("{}", counterfact[idx]['requested_rewrite']['subject'])
    msg = msg + " " + counterfact[idx]['requested_rewrite']['target_true']["str"] + "."
    msg = msg + f" Can you write a list on one line without commas of related words to the subject of the first sentence, {counterfact[idx]['requested_rewrite']['subject']}."
    return msg

In [3]:
def genResponse(msg):
    global totalTokens
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": msg}
        ],
        max_tokens=20,
        # stop=["\n", "."],
        temperature=0.8,
        presence_penalty=0.0,
    )
    totalTokens += response["usage"]["total_tokens"]
    return response

In [4]:
punctuation = [".", ",", "!", "?", ";", ":", "-", "(", ")", "[", "]", "{", "}", "<", ">"]
def postProcess(words):
    for punc in punctuation:
        words = words.replace(punc, " ")
    words = words.split(" ")
    words = [word for word in words if word != ""]
    return words

In [5]:
i = len(related)
print(counterfact[i])
msg = genMsg(i)
print(msg)

IndexError: list index out of range

In [6]:
# response = genResponse(msg)

In [7]:
# print(response["choices"][0]["message"]["content"])
# print(postProcess(response["choices"][0]["message"]["content"]))
# print(response["choices"][0]["finish_reason"])
# print(response["usage"]["total_tokens"])
# print(costEstimate(totalTokens))

In [8]:
testing = False

for i in range(len(related), len(counterfact)):
    msg = genMsg(i)
    response = genResponse(msg)
    words = postProcess(response["choices"][0]["message"]["content"])
    if testing:
        print(msg)
        print(response["choices"][0]["message"]["content"])
        print(words)

    entry = counterfact[i]
    entry["relatedWords"] = words
    entry["responseRaw"] = response.to_dict()
    related.append(entry)
    with open("related.json", "w") as f:
        json.dump(related, f)

    # print(response["choices"][0]["finish_reason"])
    print(f"{i} / {len(counterfact)} Current Cost {costEstimate(totalTokens)}", end="\r")
    if costEstimate(totalTokens) > 10:
        break

    if testing:
        break

In [6]:
print(related[-1].keys())
print(related[-1]["relatedWords"])
print(len(related))

dict_keys(['case_id', 'pararel_idx', 'requested_rewrite', 'paraphrase_prompts', 'neighborhood_prompts', 'attribute_prompts', 'generation_prompts', 'relatedWords', 'responseRaw'])
['actor', 'theatre', 'film', 'stage', 'performance', 'drama', 'comedy', 'tragedy', 'audition', 'rehearsal', 'script', 'character', 'role', 'director', 'producer', 'cinematographer', 'screenplay', 'set', 'design']
21919
